In [47]:
import pandas as pd
import numpy as np
from random import sample
from pandas import *
from collections import Counter
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

import os, sys

# Libraries from Alan
from __future__ import division
import json
from matplotlib.pyplot import *
from glob import glob
from __future__ import print_function

import requests
#import tmy_ids

# Show Plots in the Notebook
%matplotlib inline

# Make Plots larger
rcParams['figure.figsize']= (10, 6)
rcParams['font.size'] = 14

%matplotlib inline

In [50]:
class BinWxData:

# Width of the temperature bins in degrees F.  Should be an even 
# number so midpoint is an integer

    BIN_WIDTH = 2

# Minimum temperature below which no bins are created.  Should be an
# integer.

    MIN_TEMP = 50

    def __init__(self):
        self.reset()

    def reset(self):
        '''Resets the counters and accumulators to start a new binning
        process.
        '''
        self.hours = 0         # tracks total hours of data analyzed
        self.totSolar = 0.0    # tracks total solar for all hours
        self.temp = Counter()  # number of hours in each temperature bin
        self.solar = Counter() # solar occuring in each temperature bin

    def add(self, tempVal, solarVal):
    
        '''Adds an hour of temp/solar data to the analysis.
        'tempVal' is the temperature during the hour, deg F
        'solarVal' is the solar during the hours, any units
        '''
        self.hours += 1
        self.totSolar += solarVal

    # do not bin up if below the minimum temperature
        minT = BinWxData.MIN_TEMP
        if tempVal < minT:
            return

    # determine the temperature bin, and add a count to that
    # bin; add the solar to the solar Counter for that bin.
        bw = BinWxData.BIN_WIDTH
        bin = int((tempVal - minT) // bw) * bw + minT + bw // 2
        self.temp[bin] += 1
        self.solar[bin] += solarVal

    def results(self):
        '''Returns the results of the binning process in the form
        of a string.  In the string, each bin is separated by a space
        and information within the one bin is separated by commas. An
        example is:  '71,23,2.560 73,14,2.923'
        which gives information for two bins.  For each bin, the first
        value is the midpoint temperature of the bin, e.g. for a bin
        width of 2 deg F, a value of 71 means the bin is  
        70 F <= temperature < 72 F.
        The second number describing the bin is the number of hours that
        fall into the indicated bin.  The third number describing the bin
        is the ratio of solar power (W or Btu/hour) occuring for the bin
        hours relative to the average solar power for the time period 
        analyzed, averaging all of the hours in the time period, including 
        those that fall outside of the bins.
        '''
        avgSolar = self.totSolar / self.hours
        tempBins = sorted(self.temp)
        result = ''
        for b in tempBins:
            if len(result):
                result += " "
            binCount = self.temp[b]
            result += '%s,%s,%.3f' % (b, binCount, self.solar[b] / binCount / avgSolar)
        return result


In [51]:
def test():
   '''
   Test Routine to exercise BinWxData class.
   '''
   bw = BinWxData()
   bw.add(70, 120)
   bw.add(23.0, 10)
   bw.add(71.5, 140)
   bw.add(80, 200)
   print(bw.results())
   bw.reset()
   bw.add(73, 400)
   bw.add(45, 0)
   print(bw.results())

In [52]:
test()

71,2,1.106 81,1,1.702
73,1,2.000


In [53]:
# Set of TMY3 files that exactly match one of the 52 NOAA stations with complete or standard data quality
tmy3_to_noaa = [
    (702730, 'Anchorage Intl AP', 'II', 'ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US'),
    (703980, 'Annette Island AP', 'II',  'ANNETTE WEATHER SERVICE OFFICE AIRPORT AK US'),
    (700260, 'Barrow W Post-W Rogers Arpt [NSA - ARM]', 'II', 'BARROW W POST W ROGERS AIRPORT AK US'),
    (702190, 'Bethel Airport', 'II', 'BETHEL AIRPORT AK US'),
    (701740, 'Bettles Field', 'II', 'BETTLES AIRPORT AK US'),
    (702670, 'Big Delta Allen AAF', 'II', 'BIG DELTA AIRPORT AK US'),
    (703160, 'Cold Bay Arpt', 'II', 'COLD BAY AIRPORT AK US'),
    (702610, 'Fairbanks Intl Arpt', 'II', 'FAIRBANKS INTERNATIONAL AIRPORT AK US'),
    (702710, 'Gulkana Intermediate Field', 'II',  'GULKANA AIRPORT AK US'),
    (703410, 'Homer Arpt', 'II', 'HOMER AIRPORT AK US'),
    (703810, 'Juneau Intl Arpt', 'II', 'JUNEAU INTERNATIONAL AIRPORT AK US'),
    (703260, 'King Salmon Arpt', 'II', 'KING SALMON AIRPORT AK US'),
    (703500, 'Kodiak Airport', 'II',  'KODIAK AIRPORT AK US'),
    (701330, 'Kotzebue Ralph Wein Memorial', 'II', 'KOTZEBUE RALPH WEIN MEMORIAL AIRPORT AK '),
    (702310, 'McGrath Arpt', 'II', 'MCGRATH AIRPORT AK US'),
    (702000, 'Nome Municipal Arpt', 'II', 'NOME MUNICIPAL AIRPORT AK US'),
    (703080, 'St Paul Island Arpt', 'II', 'ST PAUL ISLAND AIRPORT AK US'),
    (702510, 'Talkeetna State Arpt', 'II', 'TALKEETNA AIRPORT AK US'),
    (703610, 'Yakutat State Arpt', 'II', 'YAKUTAT AIRPORT AK US'),
    (702986, 'Big River Lake', 'II', 'BIG RIVER LAKES AK US'),
    (702960, 'Cordova', 'II','CORDOVA M K SMITH AIRPORT AK US'),
    (704890, 'Dutch Harbor', 'II', 'DUTCH HARBOR AK US'),
    (702650, 'Fairbanks/Eielson A', 'II', 'EIELSON FIELD AK US'),
    
    #'Homer Arpt' is 7 miles away from Homer 8 NW.  The airport is on the coast and the NW site is inland.
    # Auke bay:  do we need this one?
    # Alyeska: don't think we need it
    # Cannery: middle of nowhere
    # College 5 NW, College, North Pole, University Experiment Station are unnecessary since we have Fairbanks station
    # Cooper landing: no nearby Tmy3
    # Eagle airport - nothing nearby
    # Glen alps : no station
    # Kitoi bay : nothing nearby, on Kodiak island
    # Kuparuk : 28 miles from deadhorse - use deadhorse tmy station?
    # little port walter not near anything
    # Matanuska Agricultural : 6 miles from Palmer Municipal TMY3 station
    (702740, 'Palmer Municipal', 'II', 'MATANUSKA AGRICULTURAL EXPERIMENT STATION AK US'),
    # main bay: not near anything
    # Tok : nothing nearby
    # White's crossing : no nearby tmy3
    # Nabesna: nothing close
    # Sutton: 10 miles from Palmer Municipal Airport tmy3 station
    # Port Alsworth: Nothing nearby
    (703400, 'Iliamna Arpt', 'II', 'ILIAMNA AIRPORT AK US'),
    (702590, 'Kenai Municipal AP', 'II', 'KENAI MUNICIPAL AIRPORT AK US'),
    (702910, 'Northway Airport', 'II', 'NORTHWAY AIRPORT AK US'),
    (702770, 'Seward', 'II', 'SEWARD AIRPORT AK US'),
    (703710, 'Sitka Japonski AP', 'II', 'SITKA AIRPORT AK US'),
    (701780, 'Tanana Ralph M Calhoun Mem AP', 'II', 'TANANA CALHOUN MEMORIAL AIRPORT AK US'),
    (702750, 'Valdez Wso', 'II', 'VALDEZ WEATHER SERVICE OFFICE AK US'),
    (703870, 'Wrangell', 'II', 'WRANGELL AIRPORT AK US')
]

In [54]:
# Sort main weather stations
tmy3_to_noaa.sort(key=lambda x: x[1])
tmy3_to_noaa

[(702730,
  'Anchorage Intl AP',
  'II',
  'ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US'),
 (703980,
  'Annette Island AP',
  'II',
  'ANNETTE WEATHER SERVICE OFFICE AIRPORT AK US'),
 (700260,
  'Barrow W Post-W Rogers Arpt [NSA - ARM]',
  'II',
  'BARROW W POST W ROGERS AIRPORT AK US'),
 (702190, 'Bethel Airport', 'II', 'BETHEL AIRPORT AK US'),
 (701740, 'Bettles Field', 'II', 'BETTLES AIRPORT AK US'),
 (702670, 'Big Delta Allen AAF', 'II', 'BIG DELTA AIRPORT AK US'),
 (702986, 'Big River Lake', 'II', 'BIG RIVER LAKES AK US'),
 (703160, 'Cold Bay Arpt', 'II', 'COLD BAY AIRPORT AK US'),
 (702960, 'Cordova', 'II', 'CORDOVA M K SMITH AIRPORT AK US'),
 (704890, 'Dutch Harbor', 'II', 'DUTCH HARBOR AK US'),
 (702610,
  'Fairbanks Intl Arpt',
  'II',
  'FAIRBANKS INTERNATIONAL AIRPORT AK US'),
 (702650, 'Fairbanks/Eielson A', 'II', 'EIELSON FIELD AK US'),
 (702710, 'Gulkana Intermediate Field', 'II', 'GULKANA AIRPORT AK US'),
 (703410, 'Homer Arpt', 'II', 'HOMER AIRPORT AK US'),
 (7034

In [55]:
# Create blank dataframe so that stuff can be appended
tmy_bindata = pd.DataFrame()

for ids, tmy_name, tmy_class, noaa_name, in tmy3_to_noaa:
 
    # Read the TMY3 file into a Pandas DataFrame direct from the NREL site
    tmy = pd.read_csv('http://rredc.nrel.gov/solar/old_data/nsrdb/1991-2005/data/tmy3/%sTYA.CSV' % ids, skiprows=1)
    # Create column with noaa station name
    tmy['STATION_NAME'] = noaa_name
    tmy_bindata = tmy_bindata.append(tmy)
    
tmy_bindata.head()

,Date (MM/DD/YYYY),Time (HH:MM),ETR (W/m^2),ETRN (W/m^2),GHI (W/m^2),GHI source,GHI uncert (%),DNI (W/m^2),DNI source,DNI uncert (%),...,Alb source,Alb uncert (code),Lprecip depth (mm),Lprecip quantity (hr),Lprecip source,Lprecip uncert (code),PresWth (METAR code),PresWth source,PresWth uncert (code),STATION_NAME
0,01/01/1976,01:00,0,0,0,1,0,0,1,0,...,?,0,0,1,D,9,45,C,8,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US
1,01/01/1976,02:00,0,0,0,1,0,0,1,0,...,?,0,0,1,D,9,45,C,8,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US
2,01/01/1976,03:00,0,0,0,1,0,0,1,0,...,?,0,0,1,D,9,10,C,8,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US
3,01/01/1976,04:00,0,0,0,1,0,0,1,0,...,?,0,0,1,D,9,45,C,8,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US
4,01/01/1976,05:00,0,0,0,1,0,0,1,0,...,?,0,0,1,D,9,45,C,8,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US


In [57]:
# convert Celsius to Fahrenheit
tmy_bindata['Temp (F)'] = tmy_bindata['Dry-bulb (C)'] * 9 / 5 + 32


# Shorten tmy dataset to only required variables
tmy_bindata = tmy_bindata[['STATION_NAME', 'Date (MM/DD/YYYY)','Time (HH:MM)', 'Temp (F)','GHI (W/m^2)', 'DNI (W/m^2)']]
tmy_bindata.head()

,STATION_NAME,Date (MM/DD/YYYY),Time (HH:MM),Temp (F),GHI (W/m^2),DNI (W/m^2)
0,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US,01/01/1976,01:00,21.02,0,0
1,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US,01/01/1976,02:00,19.94,0,0
2,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US,01/01/1976,03:00,21.02,0,0
3,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US,01/01/1976,04:00,19.04,0,0
4,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US,01/01/1976,05:00,19.04,0,0


In [62]:
tmy_bindata['hour_8760'] = tmy_bindata.index

In [65]:
# Changes date to pandas DateTime format
tmy_bindata['Date (MM/DD/YYYY)'] = to_datetime(tmy_bindata['Date (MM/DD/YYYY)'])

# Creates month column
tmy_bindata['MONTH'] = pd.DatetimeIndex(tmy_bindata['Date (MM/DD/YYYY)']).month

tmy_bindata.head()

,STATION_NAME,Date (MM/DD/YYYY),Time (HH:MM),Temp (F),GHI (W/m^2),DNI (W/m^2),hour_8760,MONTH
0,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US,1976-01-01,01:00,21.02,0,0,0,1
1,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US,1976-01-01,02:00,19.94,0,0,1,1
2,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US,1976-01-01,03:00,21.02,0,0,2,1
3,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US,1976-01-01,04:00,19.04,0,0,3,1
4,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US,1976-01-01,05:00,19.04,0,0,4,1


In [66]:
# Testing code - not necessary
#station = 'YAKUTAT AIRPORT AK US'
#mask = (tmy_bindata['STATION_NAME'] == station)
#df_subset = tmy_bindata[mask]
#df_subset.head()

,STATION_NAME,Date (MM/DD/YYYY),Time (HH:MM),Temp (F),GHI (W/m^2),DNI (W/m^2),hour_8760,MONTH
0,YAKUTAT AIRPORT AK US,1990-01-01,01:00,21.92,0,0,0,1
1,YAKUTAT AIRPORT AK US,1990-01-01,02:00,21.92,0,0,1,1
2,YAKUTAT AIRPORT AK US,1990-01-01,03:00,21.02,0,0,2,1
3,YAKUTAT AIRPORT AK US,1990-01-01,04:00,21.02,0,0,3,1
4,YAKUTAT AIRPORT AK US,1990-01-01,05:00,21.02,0,0,4,1


In [73]:
fout = open('results.txt', 'w')
flibOut = open('lib_xfer.txt', 'w')

for ids, tmy_name, tmy_class, noaa_name, in tmy3_to_noaa:
    
    # Create a mask to look at one station at a time
    mask = (tmy_bindata['STATION_NAME'] == noaa_name)
    df_subset = tmy_bindata[mask]
    
    # Set variable to station name
    akWxID = noaa_name
    
    #create a bin weather analyzer
    bw = BinWxData()

    hour = 0           # Start with the first row in the dataframe
    lastMo = 1            # tracks month of last data line
    while hour < len(df_subset['hour_8760']):
        hour_row = df_subset.query("hour_8760 == @hour") #Gets the row for the current hour
        mo = hour_row.iloc[0]['MONTH']   #Gets the month of the current hour
        if mo != lastMo:

            # have moved to a new month, output results from completed month
            print('Month %d\t%s' % (lastMo, bw.results()), file=fout)

            # add to the AkWarm library transfer file
            print('%s\t%s\t%s' % (akWxID, lastMo, bw.results()), file=flibOut)

            bw.reset()
            lastMo = mo
         
        dbTemp = float(hour_row.iloc[0]['Temp (F)'])

        # For solar, we weight direct normal solar by 40% and diffuse
        # horizontal by 50% (vertical surfaces have 50% view of sky). 
        # The direct solar weighting ought to reflect the % of direct 
        # solar received by a typical fenestration in the building, less
        # than 100% because windows are not directly pointed at the sun
        # all of the time.  These two percentages do NOT need to add to 100%,
        # since the ultimate solar value resulting from this process is a ratio
        # of solar power in a particular bin to the average solar power.
        # These factors are a judgement call, but the results do not change
        # tremendously varying the direct normal weighting from 0.2 to 0.8.
        solar = 0.4 * float(hour_row.iloc[0]['DNI (W/m^2)']) + 0.5 * float(hour_row.iloc[0]['GHI (W/m^2)'])
   
        # add the data to the analyzer
        bw.add(dbTemp, solar)

        # Use the next row in the DataFrame
        hour += 1

    # print out the month just completed
    print('Month %d\t%s' % (lastMo, bw.results()), file=fout)

    # add to the AkWarm library transfer file as well
    print('%s\t%s\t%s' % (akWxID, lastMo, bw.results()), file=flibOut)